In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import torch
from matplotlib.ticker import MaxNLocator
from tqdm import tqdm

import os
import sys

# Set up path to import from src
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

# Import pokie from pokie.py
from pokie import pokie, get_device

# Section 3.4: Strong lensing background source reconstruction: detecting prior distribution shifts

TO DO

# Data Set Up

In [3]:
# load in the .pt files in Downloads/samples
truth = torch.load('/Users/sammysharief/Downloads/samples/true.pt', map_location=torch.device('cpu'))
posterior4 = torch.load('/Users/sammysharief/Downloads/samples/posterior4.pt', map_location=torch.device('cpu'))
posterior3 = torch.load('/Users/sammysharief/Downloads/samples/posterior3.pt', map_location=torch.device('cpu'))
posterior2 = torch.load('/Users/sammysharief/Downloads/samples/posterior2.pt', map_location=torch.device('cpu'))
posterior1 = torch.load('/Users/sammysharief/Downloads/samples/posterior1.pt', map_location=torch.device('cpu'))
posterior0 = torch.load('/Users/sammysharief/Downloads/samples/posterior0.pt', map_location=torch.device('cpu'))
observations = torch.load('/Users/sammysharief/Downloads/samples/observations.pt', map_location=torch.device('cpu'))

# Load Data and Normalize

In [ ]:
curr_num_runs = 100

x_obs = truth
y_obs = observations
post_samp0 = posterior0
post_samp1 = posterior1
post_samp2 = posterior2
post_samp3 = posterior3

# Find max and min for truth, posterior4, posterior3, posterior2, posterior1, posterior0, observations
truth_max = torch.max(truth)
truth_min = torch.min(truth)

posterior4_max = torch.max(posterior4)
posterior4_min = torch.min(posterior4)

posterior3_max = torch.max(posterior3)
posterior3_min = torch.min(posterior3)

posterior2_max = torch.max(posterior2)
posterior2_min = torch.min(posterior2)

posterior1_max = torch.max(posterior1)
posterior1_min = torch.min(posterior1)

posterior0_max = torch.max(posterior0)
posterior0_min = torch.min(posterior0)

observations_max = torch.max(observations)
observations_min = torch.min(observations)

# Identify the max and min of all the data
max_data = truth_max
min_data = truth_min

# Normalize the data based on max_data and min_data
truth = (truth - min_data) / (max_data - min_data)
posterior4 = (posterior4 - min_data) / (max_data - min_data + 1e-10)
posterior3 = (posterior3 - min_data) / (max_data - min_data + 1e-10)
posterior2 = (posterior2 - min_data) / (max_data - min_data + 1e-10)
posterior1 = (posterior1 - min_data) / (max_data - min_data + 1e-10)
posterior0 = (posterior0 - min_data) / (max_data - min_data + 1e-10)

print(f'Truth max: {torch.max(truth)} and min: {torch.min(truth)}')
print(f'Posterior4 max: {torch.max(posterior4)} and min: {torch.min(posterior4)}')
print(f'Posterior3 max: {torch.max(posterior3)} and min: {torch.min(posterior3)}')
print(f'Posterior2 max: {torch.max(posterior2)} and min: {torch.min(posterior2)}')
print(f'Posterior1 max: {torch.max(posterior1)} and min: {torch.min(posterior1)}')
print(f'Posterior0 max: {torch.max(posterior0)} and min: {torch.min(posterior0)}')

# Reshape the Truth 
truth = truth.reshape(16, 64 * 64 * 3)

# Reshape all posteriors from 16,3,64,64 to 16,64,3x64x64
posterior3_reshape = posterior3.reshape(16, 64, 64 * 64 * 3)
posterior2_reshape = posterior2.reshape(16, 64, 64 * 64 * 3)
posterior1_reshape = posterior1.reshape(16, 64, 64 * 64 * 3)
posterior0_reshape = posterior0.reshape(16, 64, 64 * 64 * 3)

# Make truth, posterior0, posterior1, posterior2, posterior3, posterior4, observations into numpy arrays
truth = truth.numpy()
posterior3_reshape = posterior3_reshape.numpy()
posterior2_reshape = posterior2_reshape.numpy()
posterior1_reshape = posterior1_reshape.numpy()
posterior0_reshape = posterior0_reshape.numpy()

Truth max: 1.0 and min: 0.0
Posterior4 max: 0.8527698516845703 and min: -0.010756193660199642
Posterior3 max: 0.771514356136322 and min: -0.006224202457815409
Posterior2 max: 0.6889628767967224 and min: -0.0005897657829336822
Posterior1 max: 1.0044115781784058 and min: -0.013637484982609749
Posterior0 max: 1.038083791732788 and min: -2.8896414733026177e-05


In [5]:
# Identify your device (CUDA > MPS > CPU)
device = get_device()
print("Using device:", device)

posterior = [posterior0_reshape, posterior1_reshape, posterior2_reshape, posterior3_reshape]

# Convert to torch Tensors on the chosen device
posterior   = torch.tensor(posterior, dtype=torch.float32, device=device)
truth = torch.tensor(truth,   dtype=torch.float32, device=device)

probability, quality, n_over_N_vals = pokie(
    truth, posterior, num_runs=curr_num_runs
)

# Convert results, calibrated, n_over_N_vals back to numpy arrays
probability = probability.cpu().numpy()
quality = quality.cpu().numpy()
n_over_N_vals = n_over_N_vals.cpu().numpy()

l_values = np.linspace(0, len(probability) - 1, len(probability))
l_values = np.array(l_values)

print('\Modes:', l_values)
print("Pokie Probability:", probability)
print("Pokie Score:", quality)

Using device: mps


/var/folders/jv/k_v96nss5g52chcq2phrbl240000gn/T/ipykernel_63380/2162387673.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1702400227158/work/torch/csrc/utils/tensor_new.cpp:264.)
  posterior   = torch.tensor(posterior, dtype=torch.float32, device=device)
Pokie MC runs: 100%|██████████| 100/100 [00:01<00:00, 77.87it/s]


\Modes: [0. 1. 2. 3.]
Pokie Probability: [0.61895585 0.2553129  0.0865976  0.03913368]
Pokie Score: [0.6632617 0.5673438 0.5228418 0.5103223]
